# 欧洲大事年表项目

## 项目概述

本项目使用LangChain进行数据处理，SQLite存储历史数据，d3.js进行可视化展示。

**数据范围**: 公元前1000年到2026年

**核心功能**:
1. 抓取权威历史网站数据
2. 使用LangChain处理和结构化数据
3. 存储到SQLite数据库
4. 支持多维查询和时间轴浏览
5. 生成可复用的事件年表生成程序

**数据库设计**:
- 事件表 (events): 存储具体历史事件
- 时期表 (periods): 存储历史时期（连续或独立）
- 地域字段: 支持多个地区的历史数据

## 安装项目依赖

运行以下命令安装必要的Python包：

In [ ]:
%pip install openai langchain langchain-openai langchain-community pandas python-dotenv sqlalchemy

## 导入必要的库

In [1]:
import os
import json
import dotenv
# from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from typing import List, Dict, Optional

# LangChain imports
from langchain_openai import ChatOpenAI
# from langchain_community.chat_models import ChatOpenAI  # Fallback for older installations
from langchain_core.prompts import ChatPromptTemplate
# Output parsers: Use PydanticOutputParser or JsonOutputParser from langchain_core.output_parsers
# from langchain_core.output_parsers import PydanticOutputParser, JsonOutputParser
# Tools: Use Tool from langchain_core.tools or @tool decorator
# from langchain_core.tools import Tool

# Database imports
from sqlalchemy import create_engine, text

print("所有依赖导入成功！")

所有依赖导入成功！


## 数据库表结构设计

### 事件表 (events)

| 字段名 | 类型 | 说明 |
|--------|------|------|
| id | INTEGER PRIMARY KEY AUTOINCREMENT | 主键 |
| event_name | VARCHAR(255) NOT NULL | 事件名称 |
| start_year | INTEGER NOT NULL | 开始年份（负数表示公元前） |
| end_year | INTEGER | 结束年份（可选，null表示单年事件） |
| key_figures | TEXT | 关键人物（JSON数组格式） |
| description | TEXT | 事件概述 |
| impact | TEXT | 事件影响 |
| category | VARCHAR(100) | 事件分类（政治变革、科技突破、军事和运动、经济、文化艺术等） |
| region | VARCHAR(100) | 地域（欧洲、中国等） |
| importance_level | INTEGER | 重要程度（1-10，用于筛选重要事件） |
| source | TEXT | 数据来源URL |
| created_at | DATETIME DEFAULT CURRENT_TIMESTAMP | 创建时间 |

### 时期表 (periods)

| 字段名 | 类型 | 说明 |
|--------|------|------|
| id | INTEGER PRIMARY KEY AUTOINCREMENT | 主键 |
| period_name | VARCHAR(255) NOT NULL | 时期名称 |
| start_year | INTEGER NOT NULL | 开始年份 |
| end_year | INTEGER NOT NULL | 结束年份 |
| period_type | VARCHAR(50) NOT NULL | 时期类型（continuous连续/independent独立） |
| description | TEXT | 时期描述 |
| region | VARCHAR(100) | 地域 |
| created_at | DATETIME DEFAULT CURRENT_TIMESTAMP | 创建时间 |

## 配置数据库连接

使用SQLite数据库文件。数据将存储在项目目录下的data.db文件中。

In [2]:
# 数据库连接字符串 (SQLite)
DB_CONNECTION = "sqlite:///data.db"

# 使用SQLAlchemy创建引擎
engine = create_engine(DB_CONNECTION)

print("数据库引擎创建成功！")

数据库引擎创建成功！


## 创建数据库表

In [12]:
# 创建表的SQL语句 (SQLite)
create_events_table = """
CREATE TABLE IF NOT EXISTS events (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    event_name VARCHAR(255) NOT NULL,
    start_year INTEGER NOT NULL,
    end_year INTEGER,
    key_figures TEXT,
    description TEXT,
    impact TEXT,
    category VARCHAR(100),
    region VARCHAR(100),
    importance_level INTEGER DEFAULT 5,
    source TEXT,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
);
"""

create_periods_table = """
CREATE TABLE IF NOT EXISTS periods (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    period_name VARCHAR(255) NOT NULL,
    start_year INTEGER NOT NULL,
    end_year INTEGER NOT NULL,
    period_type VARCHAR(50) NOT NULL,
    description TEXT,
    region VARCHAR(100),
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
);
"""

# 创建索引以提高查询性能 (SQLite需要逐条执行)
create_indexes = [
    "CREATE INDEX IF NOT EXISTS idx_events_start_year ON events(start_year);",
    "CREATE INDEX IF NOT EXISTS idx_events_region ON events(region);",
    "CREATE INDEX IF NOT EXISTS idx_events_category ON events(category);",
    "CREATE INDEX IF NOT EXISTS idx_events_importance ON events(importance_level);",
    "CREATE INDEX IF NOT EXISTS idx_periods_start_year ON periods(start_year);",
    "CREATE INDEX IF NOT EXISTS idx_periods_region ON periods(region);",
    "CREATE INDEX IF NOT EXISTS idx_periods_type ON periods(period_type);"
]

# 执行建表
with engine.connect() as conn:
    conn.execute(text(create_events_table))
    conn.execute(text(create_periods_table))
    for index_sql in create_indexes:
        conn.execute(text(index_sql))
    conn.commit()

print("数据库表和索引创建成功！")

数据库表和索引创建成功！


## 验证表结构

In [3]:
# 查看表结构 (SQLite)
with engine.connect() as conn:
    result = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table'"))
    tables = [row[0] for row in result]
    
print("数据库中的表:", tables)

# 查看events表结构
if 'events' in tables:
    with engine.connect() as conn:
        result = conn.execute(text("PRAGMA table_info(events)"))
        print("\nevents表结构:")
        for row in result:
            print(f"  {row[1]}: {row[2]} (nullable: {'YES' if row[3] else 'NO'}, default: {row[4]})")

# 查看periods表结构
if 'periods' in tables:
    with engine.connect() as conn:
        result = conn.execute(text("PRAGMA table_info(periods)"))
        print("\nperiods表结构:")
        for row in result:
            print(f"  {row[1]}: {row[2]} (nullable: {'YES' if row[3] else 'NO'}, default: {row[4]})")

数据库中的表: ['events', 'sqlite_sequence', 'periods']

events表结构:
  id: INTEGER (nullable: NO, default: None)
  event_name: VARCHAR(255) (nullable: YES, default: None)
  start_year: INTEGER (nullable: YES, default: None)
  end_year: INTEGER (nullable: NO, default: None)
  key_figures: TEXT (nullable: NO, default: None)
  description: TEXT (nullable: NO, default: None)
  impact: TEXT (nullable: NO, default: None)
  category: VARCHAR(100) (nullable: NO, default: None)
  region: VARCHAR(100) (nullable: NO, default: None)
  importance_level: INTEGER (nullable: NO, default: 5)
  source: TEXT (nullable: NO, default: None)
  created_at: DATETIME (nullable: NO, default: CURRENT_TIMESTAMP)

periods表结构:
  id: INTEGER (nullable: NO, default: None)
  period_name: VARCHAR(255) (nullable: YES, default: None)
  start_year: INTEGER (nullable: YES, default: None)
  end_year: INTEGER (nullable: YES, default: None)
  period_type: VARCHAR(50) (nullable: YES, default: None)
  description: TEXT (nullable: NO, de

## 配置LangChain

LangChain需要OpenAI API密钥才能使用大模型功能。

### 配置方式

**方法1: 设置环境变量（推荐）**
```bash
export OPENAI_API_KEY="your-api-key-here"
```

**方法2: 在代码中设置**
```python
os.environ["OPENAI_API_KEY"] = "your-api-key-here"
```

### LangChain Tools配置说明

LangChain tools允许将自定义函数包装为可被大模型调用的工具。在后续步骤中，我们将创建以下工具：

1. **数据抓取工具**: 抓取网页内容
2. **数据验证工具**: 验证数据格式和完整性
3. **数据库插入工具**: 将数据存入SQLite
4. **历史分析工具**: 分析事件影响和分类

这些工具将通过LangChain的`Tool`类进行封装，然后传递给大模型使用。

## 初始化LangChain大模型

如果已配置OpenAI API密钥，可以初始化Chat模型：

In [4]:
dotenv.load_dotenv(override=True)
# 检查API密钥
if "OPENAI_API_KEY" not in os.environ:
    print("警告: OPENAI_API_KEY环境变量未设置")
    print("请设置API密钥: os.environ['OPENAI_API_KEY'] = 'your-key-here'")
    llm = None
else:
    # 初始化Chat模型
    llm = ChatOpenAI(
        model=os.environ["OPENAI_MODEL"],
        temperature=0.3,
        api_key=os.environ["OPENAI_API_KEY"],
        base_url=os.environ["OPENAI_BASE_URL"]
    )
    print("LangChain模型初始化成功！")

/var/folders/6d/q0fz70cd7d9_gdxq5ctzkzn00000gn/T/ipykernel_10997/1575897353.py:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


LangChain模型初始化成功！


In [ ]:
response = llm.invoke("你好，请介绍一下自己。")
print(response.content)

## 清空测试数据

在开始生成真实数据前，先清空测试数据。

## 数据采集策略

**分阶段采样+重要性过滤**：将-1000到2026分为6个历史阶段，按不同密度采样，只保留重要事件（importance≥6）。

### 6个历史阶段：
1. **古典时期** (-1000到500)：每100年采样一次
2. **中世纪** (501到1500)：每50年采样一次
3. **近代早期** (1501到1800)：每25年采样一次
4. **19世纪** (1801到1900)：每10年采样一次
5. **20世纪** (1901到2000)：每5年采样一次
6. **21世纪** (2001到2026)：每年采样

预计API请求次数：约80次
预计数据量：约400-500个高质量事件

In [5]:
# 确保导入text函数
from sqlalchemy import text

# 清空events表 (SQLite)
with engine.connect() as conn:
    conn.execute(text("DELETE FROM events"))
    conn.execute(text("DELETE FROM sqlite_sequence WHERE name='events'"))
    conn.commit()
    print("events表已清空！")

# 清空periods表 (SQLite)
with engine.connect() as conn:
    conn.execute(text("DELETE FROM periods"))
    conn.execute(text("DELETE FROM sqlite_sequence WHERE name='periods'"))
    conn.commit()
    print("periods表已清空！")

# 验证清空结果
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM events"))
    events_count = result.fetchone()[0]
    result = conn.execute(text("SELECT COUNT(*) FROM periods"))
    periods_count = result.fetchone()[0]
    print(f"当前数据: events={events_count}, periods={periods_count}")

events表已清空！
periods表已清空！
当前数据: events=0, periods=0


In [4]:
# 导入TimelineGenerator
import dotenv
import os
from timeline_generator import TimelineGenerator

dotenv.load_dotenv(override=True)
model=os.environ["OPENAI_MODEL"]
api_key=os.environ["OPENAI_API_KEY"]
base_url=os.environ["OPENAI_BASE_URL"]
print(f"{api_key},\n {base_url}, \n{model}")

sk-sllhfxkkeqpjszuuavvibpfkqwprwfyynutceciozpzlrbcx,
 https://api.siliconflow.cn/v1, 
deepseek-ai/DeepSeek-R1-0528-Qwen3-8B


## 采集欧洲历史数据（分阶段采样）

使用新的采集策略：按历史阶段分批采样，自动过滤低价值信息。

In [ ]:
# 初始化欧洲时间轴生成器
european_generator = TimelineGenerator(region="European",
                                    llm_api_key=api_key,
                                    llm_base_url=base_url,
                                    llm_model=model)

# 使用新的采集策略：分阶段采样从-1000到2026
print("开始采集欧洲历史数据...")
print("预计需要约80次API请求，请耐心等待...")

result = european_generator.scrape_full_timeline(
    classical_years=100,      # 古典时期：每100年
    medieval_years=50,         # 中世纪：每50年
    early_modern_years=25,     # 近代早期：每25年
    nineteenth_century_years=10, # 19世纪：每10年
    twentieth_century_years=5,    # 20世纪：每5年
    twenty_first_century_years=1, # 21世纪：每年
    min_importance=6,          # 只保留重要事件
    progress_callback=lambda x: print(f"  {x}")
)

print(f"\n采集完成！")
print(f"总事件数: {result['events']}")
print(f"时间范围: -1000到2026（完整覆盖）")

## 清空中国历史数据

在重新采集中国历史数据前，先清空已有的中国历史数据。

In [8]:
# 清空中国地区的事件数据
with engine.connect() as conn:
    conn.execute(text("DELETE FROM events WHERE region='Chinese'"))
    # conn.execute(text("DELETE FROM sqlite_sequence WHERE name='events'"))
    conn.commit()
    print("中国历史事件数据已清空！")

# 验证清空结果
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM events WHERE region='Chinese'"))
    events_count = result.fetchone()[0]
    print(f"当前中国历史事件数: {events_count}")


中国历史事件数据已清空！
当前中国历史事件数: 0


## 采集中国历史数据（按朝代）

使用按朝代采集的方式，对中国历史数据进行完整抓取。

In [ ]:
# 初始化中国时间轴生成器
chinese_generator = TimelineGenerator(region="Chinese",
                                    llm_api_key=api_key,
                                    llm_base_url=base_url,
                                    llm_model=model)

# 按朝代采集中国历史
print("开始采集中国历史数据...")
print("将按朝代顺序采集，共16个朝代...")
print("预计需要约16次API请求（每个朝代一次），请耐心等待...")

result = chinese_generator.scrape_from_dynasties(
    max_events_per_dynasty=20, # 每个朝代最多提取20个事件
    min_importance=5,           # 最低重要程度
    force_refresh=False,         # 使用缓存
    progress_callback=lambda x: print(f"  {x}")
)

print(f"\n采集完成！")
print(f"总事件数: {result['events']}")
print(f"时间范围: 按朝代完整覆盖中国历史")

Database tables and indexes created successfully!
开始采集中国历史数据...
将按朝代顺序采集，共16个朝代...
预计需要约16次API请求（每个朝代一次），请耐心等待...
Scraping Chinese history from 17 dynasty pages...
  Processing dynasty 夏朝 (5.9%)
Cache hit: Chinese_夏朝 (LLM)
  Processing dynasty 商朝 (11.8%)
Cache hit: Chinese_商朝 (LLM)
  Processing dynasty 周朝 (17.6%)
Cache hit: Chinese_周朝 (LLM)
  Processing dynasty 秦朝 (23.5%)
Cache hit: Chinese_秦朝 (LLM)
  Processing dynasty 汉朝 (29.4%)
Cache hit: Chinese_汉朝 (LLM)
  Processing dynasty 三国 (35.3%)
Cache hit: Chinese_三国 (LLM)
  Processing dynasty 晋朝 (41.2%)
Cache hit: Chinese_晋朝 (LLM)
  Processing dynasty 南北朝 (47.1%)
Cache hit: Chinese_南北朝 (LLM)
  Processing dynasty 隋朝 (52.9%)
Cache hit: Chinese_隋朝 (LLM)
  Processing dynasty 唐朝 (58.8%)
Cache hit: Chinese_唐朝 (LLM)
  Processing dynasty 五代十国 (64.7%)
Cache hit: Chinese_五代十国 (LLM)
  Processing dynasty 宋朝 (70.6%)
Cache hit: Chinese_宋朝 (LLM)
  Processing dynasty 元朝 (76.5%)
Cache hit: Chinese_元朝 (LLM)
  Processing dynasty 明朝 (82.4%)
Cache hit: Chinese_

KeyboardInterrupt: 

## 验证和查看生成的数据

检查数据库统计信息，并查看不同时期的事件示例。

In [ ]:
# 获取统计信息
stats = european_generator.get_statistics()
print(f"\n数据库统计:")
print(f"  总事件数: {stats['total_events']}")
print(f"  总时期数: {stats['total_periods']}")
if 'events_by_region' in stats:
    print(f"  按地区统计事件: {stats['events_by_region']}")
if 'periods_by_region' in stats:
    print(f"  按地区统计时期: {stats['periods_by_region']}")

# 查询不同时期的重要事件
print("\n=== 不同时期的重要事件 ===")

# 古典时期示例（-1000到500）
classical_events = european_generator.get_timeline(
    start_year=-500,
    end_year=0,
    min_importance=7,
    limit=5
)
print(f"\n古典时期(-500到0): {len(classical_events)}个事件")
for event in classical_events[:3]:
    print(f"  - {event['event_name']} ({event['start_year']}) - {event.get('category', 'N/A')}")

# 中世纪示例（501到1500）
medieval_events = european_generator.get_timeline(
    start_year=1000,
    end_year=1200,
    min_importance=7,
    limit=5
)
print(f"\n中世纪(1000到1200): {len(medieval_events)}个事件")
for event in medieval_events[:3]:
    print(f"  - {event['event_name']} ({event['start_year']}) - {event.get('category', 'N/A')}")

# 20世纪示例（1901到2000）
twentieth_century = european_generator.get_timeline(
    start_year=1900,
    end_year=2000,
    min_importance=7,
    limit=10
)
print(f"\n20世纪(1900到2000): {len(twentieth_century)}个事件")
for event in twentieth_century[:5]:
    print(f"  - {event['event_name']} ({event['start_year']}) - {event.get('category', 'N/A')}")

# 21世纪示例（2001到2026）
twenty_first_century = european_generator.get_timeline(
    start_year=2000,
    end_year=2026,
    min_importance=7,
    limit=10
)
print(f"\n21世纪(2000到2026): {len(twenty_first_century)}个事件")
for event in twenty_first_century[:5]:
    print(f"  - {event['event_name']} ({event['start_year']}) - {event.get('category', 'N/A')}")

# 搜索示例：战争相关事件
print("\n=== 搜索示例 ===")
wars = european_generator.search_events("war", limit=10)
print(f"包含'war'的事件: {len(wars)}个")
for event in wars[:3]:
    print(f"  - {event['event_name']} ({event['start_year']})")

# 跨地区对比示例：1945年的中欧对比
print("\n=== 跨地区对比 ===")
cross_events = european_generator.get_cross_regional_view(
    year=1945,
    other_regions=["Chinese"],
    importance_threshold=7
)
for region, events in cross_events.items():
    print(f"{region}在1945年左右: {len(events)}个事件")
    for event in events[:3]:
        print(f"  - {event['event_name']} ({event['start_year']})")

In [ ]:
from wikipedia_scraper import WikipediaScraper

scraper = WikipediaScraper(language="en", region="European")
content = scraper.search_pages(query="Phoenicians Timeline", limit=5)
print(content)

# content = scraper.get_year_page(1945)
# print(content)

content2 = scraper.get_page_content(content[0]['pageid'])
print(content2)

[{'ns': 0, 'title': 'Timeline of ancient Greece', 'pageid': 54123, 'size': 64998, 'wordcount': 7974, 'snippet': 'is a <span class="searchmatch">timeline</span> of <span class="searchmatch">ancient</span> <span class="searchmatch">Greece</span> from its emergence around 800 BC to its subjection to the Roman Empire in 146 BC. For earlier times, see <span class="searchmatch">Greek</span> Dark Ages', 'timestamp': '2025-12-05T21:54:06Z'}, {'ns': 0, 'title': 'Timeline of ancient Greek mathematicians', 'pageid': 16266307, 'size': 21285, 'wordcount': 1464, 'snippet': 'This is a <span class="searchmatch">timeline</span> of mathematicians in <span class="searchmatch">ancient</span> <span class="searchmatch">Greece</span>. Historians traditionally place the beginning of <span class="searchmatch">Greek</span> mathematics proper to the age of Thales', 'timestamp': '2025-06-28T05:41:44Z'}, {'ns': 0, 'title': 'Outline of ancient Greece', 'pageid': 23642146, 'size': 26828, 'wordcount': 2173, 'snippet'